In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://aloft-hotels.marriott.com/locations/"
driver.get(url)
time.sleep(5)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'property'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
achotels_df = pd.DataFrame(hotel_list)
achotels_df

,name,url
0,"Aloft Beijing, Haidian",https://www.marriott.com/BJSAL
1,Aloft Dongguan Dynamic Town,https://www.marriott.com/SZXDL
2,Aloft Dongguan Songshan Lake,https://www.marriott.com/SZXAL
3,"Aloft Nanhai, Foshan",https://www.marriott.com/FUOAL
4,Aloft Guangzhou Tianhe,https://www.marriott.com/CANAL
...,...,...
198,Aloft Milwaukee Downtown,https://www.marriott.com/MKEAL
199,Aloft Bogota Airport,https://www.marriott.com/BOGAL
200,Aloft Asuncion,https://www.marriott.com/ASUAL
201,Aloft Lima Miraflores,https://www.marriott.com/LIMRA


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in achotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df      

{'name': 'Aloft Beijing, Haidian', 'url': 'https://www.marriott.com/BJSAL', 'street': 'Tower 2 No. 25 Yuanda Road, Haidian District', 'locality': 'Beijing', 'state': '', 'postalcode': '100097', 'coordinate1': '116.279068', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Celebrate your style at Aloft Beijing, Haidian, where bold loft-inspired design enlivens China's vibrant capital. We're in the heart of the high-tech and university district, with easy access to the Summer Palace, Olympic Stadium, and more! Meet and mingle with friends at our w xyz(SM) bar, grab a sweet, savory or healthy snack from re:fuel by Aloft(SM), our 24/7 pantry or play in our re:mix(SM) lounge. Plus, you can always stay connected with hotel-wide wired and wireless High-Speed Internet Access! Breeze into one of our Aloft rooms, featuring our ultra-comfortable signature bed, walk-in shower, custom amenities by Bliss® Spa and more. Our plug &amp; play connectivity station charge

{'name': 'Aloft Zhengzhou Shangjie', 'url': 'https://www.marriott.com/CGOSL', 'street': 'No.101 Zhongxin Road, Shangjie District', 'locality': 'Zhengzhou', 'state': '', 'postalcode': '450041', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/cgosl-aloft-zhengzhou-shangjie/?directPageRequest=true', 'coordinate3': '34.808105', 'coordinate4': '113.316326', 'description': "[ Celebrate your style at Aloft Zhengzhou Shangjie, located in the heart inland China's business center. We're a short stroll to endless entertainment options and a quick trip to the Shaolin Temple and the UNESCO World Heritage destination, the Luoyang Longmen Grottoes. Meet &amp; mingle with friends at our w xyz(SM) bar, grab a sweet, savory or healthy snack from re:fuel by Aloft(SM), our 24/7 pantry or play in our re:mix(SM) lounge. Plus, you can always stay connected with hotel-wide wired and wireless High Speed Internet Access! Breeze into one of our Aloft rooms, featuring our ult

{'name': 'Aloft Chengdu Shixiang Lake', 'url': 'https://www.marriott.com/CTUAL', 'street': 'Shixiang Lake Scenic Spot, Pujiang County', 'locality': 'Chengdu', 'state': '', 'postalcode': '611600', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/ctual-aloft-chengdu-shixiang-lake/?directPageRequest=true', 'coordinate3': '30.191348', 'coordinate4': '103.426104', 'description': "[ Aloft Chengdu Shixiang lake\xa0is located in shixiang lake scenic area, which is known as the oxygen bar in the forest. It is 88 km away from chengdu downtown, 68 km away from Chengdu Airport and expressway is directly access to the hotel.  It is connected with Chengdu, Ya'an，Meishan and adjacent to the provincial-level scenic spot Chaoyang Lake, Changtan Lake, Daxi Valley, Feixian Pavilion, Cherry Mountain, Xilai Town, south of Pujiang Tea Culture Natural Conservation Base which is famous for Queshe Tea and called the back garden of chengdu. ]", 'overall_rating': 'N/A', 'clea

{'name': 'Aloft Chennai OMR - IT Expressway', 'url': 'https://www.marriott.com/MAAAL', 'street': '102, Rajiv Gandhi Salai, Sholinganallur', 'locality': 'Chennai', 'state': '', 'postalcode': '600 119', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/maaal-aloft-chennai-omr-it-expressway/?directPageRequest=true', 'coordinate3': '12.899963', 'coordinate4': '80.227967', 'description': "[ Celebrate your style at Aloft Chennai OMR- IT Expressway, with loft-inspired design in India's fourth largest metropolis! We're steps from the city's many IT/ITES companies and mere minutes from the airport and city sites. Meet &amp;amp; mingle with friends at our w xyz(SM) bar, grab a sweet, savory or healthy snack from re:fuel by Aloft(SM), our 24/7 pantry, or play in our re:mix(SM) lounge. Plus, you can always stay connected with free hotel-wide wired and wireless Internet access! Breeze into one of our Aloft rooms, featuring our ultra-comfortable signature bed, an 

{'name': 'Aloft Bali Seminyak', 'url': 'https://www.marriott.com/DPSAS', 'street': 'Jalan Batu Belig No 228', 'locality': 'Seminyak', 'state': '', 'postalcode': '80361', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ With its beautiful beaches, all-night parties and vibrant restaurant scene, Seminyak has become a must for any digital nomad or influencer. In the middle of all the action is the Aloft Bali Seminyak, an industrial-inspired, lifestyle-focused hotel with a distinctly local touch. The hotel offers its own additions to the scene including the lobby’s W XYZ® bar featuring signature cocktails &amp; choices of drinks, a billiards table and on top of these, an outdoor rooftop infinity pool overlooking picturesque Batu Belig Beach. The top floor is also home to KAHUNA, an all-day dining restaurant with a beach-view terrace offering fresh, fusion-eclectic fare. Guest rooms are a place to retreat and recharge with relaxing ra

...

{'name': 'Aloft Houston Katy', 'url': 'https://www.marriott.com/HOUAT', 'street': '25330 Kingsland Boulevard', 'locality': 'Katy', 'state': 'Texas', 'postalcode': '77494', 'coordinate1': '-95.811972', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Enjoy the heart of Katy live and up close. Aloft Katy is a boutique upscale hotel, minutes from a vibrant social scene and Katy’s boardwalk. Located to major corporate companies, walking distance from Katy Mills Mall, restaurants, and minutes from Typhoon Texas Waterpark unlike anywhere else are part of the lineup outside our doors.  As inside, like-minds meet and mingle at our WXYZ Bar and Lounge over creative cocktails and live acoustics or chill over pool table in our Re:mix lounge.  Workout at our Re:charge gym equipped with all the right stuff including a beat to tone to, then take the plunge in our outdoor Splash Pool to cool down. Fast &amp; Free Wi-Fi, bright, airy hotel rooms including connecting 

{'name': 'New: Aloft Houston Shenandoah', 'url': 'https://www.marriott.com/HOUAO', 'street': '19391 David Memorial Drive', 'locality': 'Shenandoah', 'state': 'Texas', 'postalcode': '77385', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Ideally located The Woodlands, TX hotel area, the brand new Aloft Houston Shenandoah offers a more savvy experience for the modern traveler. Surrounded by shopping, entertainment and major businesses, this hotel near The Woodlands, TX offers convenient access to everything you need.Meet and mingle with friends at our W XYZ® bar, grab a sweet, savory, or healthy snack from Re:fuel by Aloft, our 24-hour pantry, or play in our Re:mix lounge. Breeze into one of our spacious rooms, featuring contemporary décor, complimentary Wi-Fi and pillowtop mattresses. Our plug and play connectivity station charges all of your electronics and links to the 55” LCD TV, maximizing work and play. Hosting an upcoming 

{'name': 'Aloft Seattle Redmond', 'url': 'https://www.marriott.com/SEAAD', 'street': '15220 NE Shen Street, Suite 150', 'locality': 'Redmond', 'state': 'Washington', 'postalcode': '98052', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/seaad-aloft-seattle-redmond/?directPageRequest=true', 'coordinate3': '47.635471', 'coordinate4': '-122.1371', 'description': "[ Celebrate your style at Aloft Seattle Redmond, a modern hotel where you'll be close to the headquarters of Microsoft, T-Mobile, SpaceX and Nintendo of America. Additionally, we offer easy-breezy access to an array of beautiful outdoor attractions, as well as the city of Bellevue. Meet and mingle with friends at our W XYZ® bar, grab a quick snack from Re:fuel by Aloft, our 24-hour pantry, or play in our Re:mix lounge. You can always stay connected with the help of our complimentary, high-speed Wi-Fi. Retreat to the warmth of our spacious hotel rooms and suites, featuring ultra-comfortable si

{'name': 'Aloft Asuncion', 'url': 'https://www.marriott.com/ASUAL', 'street': 'Avenida Aviadores del Chaco 1761', 'locality': 'Asuncion', 'state': '', 'postalcode': '', 'coordinate1': '-25.283659', 'coordinate2': '-57.568273', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Choose your vacation path at Aloft Asuncion. Our spacious hotel rooms offer deserved relaxation through plush bedding and authentic views of the city. You can also amplify your stay at our on-site restaurant or thrilling rooftop bar which contains a splash pool. All sorts of attractions are nearby, including Paseo Carmelitas and Shopping del Sol. World Trade Center Asuncion is down the street and Silvio Pettirossi International Airport is 4.3 miles away. Private event space is available for meetings or celebrations. Our Tactic Room can take many seating forms, plus it can be enhanced with available AV equipment, catering and high-speed Wi-Fi. Whether it's our venue indoors or our rooftop setting outdoo

,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,"Aloft Beijing, Haidian",https://www.marriott.com/BJSAL,"Tower 2 No. 25 Yuanda Road, Haidian District",Beijing,,100097,116.279068,N/A,N/A,N/A,"[ Celebrate your style at Aloft Beijing, Haidi...",N/A,3.9,3.5
1,"Aloft Beijing, Haidian",https://www.marriott.com/BJSAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aloft Dongguan Dynamic Town,https://www.marriott.com/SZXDL,"Dynamic Town, Fenggang",Dongguan,,523690,https://www.marriott.com/hotels/maps/travel/sz...,22.717859,114.207829,N/A,[ Celebrate your style at Aloft Dongguan Dynam...,N/A,4.4,4.3
3,Aloft Dongguan Dynamic Town,https://www.marriott.com/SZXDL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aloft Dongguan Songshan Lake,https://www.marriott.com/SZXAL,"No. 7 Xinzhu Road, Vanke Plaza, Songshan Lake ...",Dongguan,,523808,Reviews,https://www.marriott.com/hotels/maps/travel/sz...,22.929869,113.892328,[ Celebrate your style at Aloft Dongguan Songs...,N/A,4.3,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,Aloft Asuncion,https://www.marriott.com/ASUAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,Aloft Lima Miraflores,https://www.marriott.com/LIMRA,"Av. 28 de Julio N 894, Miraflores",Lima,,,Reviews,https://www.marriott.com/hotels/maps/travel/li...,-12.129264,-77.023989,[ Discover a new take on travel at Aloft Lima ...,N/A,4.8,4.5
403,Aloft Lima Miraflores,https://www.marriott.com/LIMRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,Aloft Montevideo Hotel,https://www.marriott.com/MVDAL,Victor Solino 350,Montevideo,,11300,Reviews,https://www.marriott.com/hotels/maps/travel/mv...,-34.923519,-56.158118,[ Mix it up at Aloft Montevideo Hotel in Urugu...,N/A,4.6,4.6


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name', ignore_index=True)

In [7]:
hotel_df

,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,"Aloft Beijing, Haidian",https://www.marriott.com/BJSAL,"Tower 2 No. 25 Yuanda Road, Haidian District",Beijing,,100097,116.279068,N/A,N/A,N/A,"[ Celebrate your style at Aloft Beijing, Haidi...",N/A,3.9,3.5
1,Aloft Dongguan Dynamic Town,https://www.marriott.com/SZXDL,"Dynamic Town, Fenggang",Dongguan,,523690,https://www.marriott.com/hotels/maps/travel/sz...,22.717859,114.207829,N/A,[ Celebrate your style at Aloft Dongguan Dynam...,N/A,4.4,4.3
2,Aloft Dongguan Songshan Lake,https://www.marriott.com/SZXAL,"No. 7 Xinzhu Road, Vanke Plaza, Songshan Lake ...",Dongguan,,523808,Reviews,https://www.marriott.com/hotels/maps/travel/sz...,22.929869,113.892328,[ Celebrate your style at Aloft Dongguan Songs...,N/A,4.3,4.2
3,"Aloft Nanhai, Foshan",https://www.marriott.com/FUOAL,"Boai Road, Shishan Town, Nanhai",Foshan,,528225,Reviews,https://www.marriott.com/hotels/maps/travel/fu...,23.120588,113.009624,"[ Aloft Nanhai, Foshan is conveniently located...",N/A,4.2,4.0
4,Aloft Guangzhou Tianhe,https://www.marriott.com/CANAL,"No. 365 Tianhe Bei Road, Tianhe District",Guangzhou,,510610,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,23.1422155,113.3286946,[ With a modern design aesthetic and tech-forw...,N/A,4.4,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Aloft Milwaukee Downtown,https://www.marriott.com/MKEAL,1230 Old World Third Street,Milwaukee,Wisconsin,53212,43.046077,-87.914368,N/A,N/A,[ Celebrate your style at Aloft® Hotels Milwau...,N/A,4.1,4.1
199,Aloft Bogota Airport,https://www.marriott.com/BOGAL,Avenida Calle 26 #92 - 32,Bogota,,,4.682941,-74.119467,N/A,N/A,[ Surrounded by some of the area's most celebr...,N/A,4.5,4.1
200,Aloft Asuncion,https://www.marriott.com/ASUAL,Avenida Aviadores del Chaco 1761,Asuncion,,,-25.283659,-57.568273,N/A,N/A,[ Choose your vacation path at Aloft Asuncion....,N/A,4.7,4.5
201,Aloft Lima Miraflores,https://www.marriott.com/LIMRA,"Av. 28 de Julio N 894, Miraflores",Lima,,,Reviews,https://www.marriott.com/hotels/maps/travel/li...,-12.129264,-77.023989,[ Discover a new take on travel at Aloft Lima ...,N/A,4.8,4.5


In [8]:
hotel_df.to_csv('aloft_description.csv')